In [ ]:
wdir = "/home/daniele/documents/github/ftt01/phd/projects/hydrological_forecasting/"

In [ ]:
# IMPORTs
import sys
import os
import glob
import subprocess
import pandas as pd
from dask import dataframe as dd

from shutil import rmtree

# to link the lib in py scripts as well
os.chdir(wdir)
sys.path.insert(0, os.path.join(os.path.abspath(os.getcwd()), 'lib'))
from lib import *


In [ ]:
def apply_bias_correction( fct_data ):

    fct_data['values'] = fct_data['values'] * 1.5
    return fct_data

In [ ]:
# SETUP
# basins = ['passirio', 'plan']
basins = ['plan']
variable = 'precipitation'

# init_ref = '03'
init_forecasting_hour = 9
lead_hours = 39
lag_hours = 24*7

output_types = ['mean','median','first_quantile','third_quantile']

start_date = '20210615'
end_date = '20211014'


In [ ]:
start_date = dt.datetime.strptime(start_date, '%Y%m%d')
end_date = dt.datetime.strptime(end_date, '%Y%m%d')

dates = [start_date + dt.timedelta(days=x)
         for x in range(0, (end_date-start_date).days + 1)]


In [ ]:
for output_type in output_types:

    print(output_type)

    for basin in basins:

        print(basin)
        
        fct_data_path = "/media/windows/projects/hydrological_forecasting/machine_learning/data/observed/{basin}/{variable}/daily/fullfilled/{output_type}/"
        obs_data_path = "/media/windows/projects/hydrological_forecasting/machine_learning/data/observed/{basin}/{variable}/daily/"
        output_path = "/media/windows/projects/hydrological_forecasting/machine_learning/data/observed/{basin}/{variable}/daily/unbiased/{output_type}/"

        fct_data_path = fct_data_path.format(
            basin=basin, variable=variable, output_type=output_type)
        obs_data_path = obs_data_path.format(
            basin=basin, variable=variable, output_type='mean')
        output_path = output_path.format(
            basin=basin, variable=variable, output_type=output_type)
        mkNestedDir(output_path)

        ### evaluate the bias
        # diff_fct_obs = pd.DataFrame()

        # for date in dates:

        #     # print(date)
        #     date_as_str = dt.datetime.strftime(date, '%Y%m%d')

        #     ## FORECAST DATASET ##
        #     fileName_fct = fct_data_path + date_as_str + ".csv"

        #     df_fct = pd.read_csv(fileName_fct, header=0,
        #                          index_col=0, parse_dates=True, sep=';')
        #     df_fct = df_fct['values']

        #     ## OBSERVED DATASET ##
        #     fileName_obs = obs_data_path + date_as_str + ".csv"

        #     df_obs = pd.read_csv(fileName_obs, header=0,
        #                          index_col=0, parse_dates=True, sep=';')
        #     df_obs = df_obs['values']

        #     diff_fct_obs[date_as_str] = (df_fct - df_obs).values

        #     # print(diff_fct_obs)

        ### apply the bias correction
        for date in dates:

            # print(date)

            date_as_str = dt.datetime.strftime(date, '%Y%m%d')
            # print(date_as_str)

            current_fct_data = fct_data_path + date_as_str + ".csv"
            # print( current_fct_data )

            fct_data = pd.read_csv(
                current_fct_data, parse_dates=True, header=0, sep=';')
            fct_data.set_index('datetime', inplace=True)

            if date != start_date:
                fct_data_unbiased = apply_bias_correction(fct_data)
            else:
                fct_data_unbiased = fct_data
                
            fct_data_unbiased.to_csv(
                output_path + date_as_str + '.csv', sep=';')